In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import getpass
import pdvega
import time
from configobj import ConfigObj

In [2]:
# Create a posgresql database connection using settings from config file 
config='../db/config.ini'
# connection info
conn_info = dict()
if os.path.isfile(config):
    config = ConfigObj(config)
    conn_info["sqluser"] = config['username']
    conn_info["sqlpass"] = config['password']
    conn_info["sqlhost"] = config['host']
    conn_info["sqlport"] = config['port']
    conn_info["dbname"] = config['dbname']
    conn_info["schema_name"] = config['schema_name']
else:
    conn_info["sqluser"] = 'postgres'
    conn_info["sqlpass"] = '243'
    conn_info["sqlhost"] = 'localhost'
    conn_info["sqlport"] = 5432
    conn_info["dbname"] = 'eicu'
    conn_info["schema_name"] = 'public,eicu'
    
# Connect to the eICU database
print('Database: {}'.format(conn_info['dbname']))
print('Username: {}'.format(conn_info["sqluser"]))
if conn_info["sqlpass"] == '243':
    # try connecting without password, i.e. peer or OS authentication
    try:
        if (conn_info["sqlhost"] == 'localhost') & (conn_info["sqlport"]=='5432'):
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   user=conn_info["sqluser"])            
        else:
            con = psycopg2.connect(dbname=conn_info["dbname"],
                                   host=conn_info["sqlhost"],
                                   port=conn_info["sqlport"],
                                   user=conn_info["sqluser"])
    except:
        conn_info["sqlpass"] = getpass.getpass('Password: ')

        con = psycopg2.connect(dbname=conn_info["dbname"],
                               host=conn_info["sqlhost"],
                               port=conn_info["sqlport"],
                               user=conn_info["sqluser"],
                               password=conn_info["sqlpass"])
query_schema = 'set search_path to ' + conn_info['schema_name'] + ';'

Database: eicu
Username: postgres
Password: ········


In [3]:
#import patient data, if not exsit allpatientdata.csv file than please excute the patientdata.ipynb file first
df=pd.read_csv(os.getcwd()+'\\allpatientfile.csv')

In [4]:
#可調整輸出的方式
#adjust the extract data into output format
def Featrue_all_value_with_day(ft_name, day, df_intakeoutput):
    temp_1 = [-0.001]
    for j in range(0,len(df_intakeoutput.index)):
        if df_intakeoutput['date'][j] == day: 
            temp_1.append(df_intakeoutput[ft_name][j])
    if len(temp_1) != 1:
        temp_1.pop(0)  
    temp = 0
    count_space = 0
    for i in range(len(temp_1)):
        if temp_1[i] == -0.001:
            count_space = count_space + 1
            continue
        else:
            temp = temp + temp_1[i]
    if len(temp_1) == 1:
        if temp_1[0] == -0.001:
            temp = -0.001
    return temp

In [5]:
def Intakeoutput(patientunitstayid, Firstday, lenofstay, th, df_IO, ftName_0):
    query = query_schema + """
    select *
    from intakeoutput
    where patientunitstayid = '{}'
    order by intakeoutputoffset
    """.format(int(patientunitstayid))
    df_intakeoutput = pd.read_sql_query(query, con)
    df_intakeoutput = df_intakeoutput[['patientunitstayid','intakeoutputoffset','intaketotal','outputtotal','dialysistotal','nettotal']]
    
    #計算該row是第幾天
    #calculate event time by days
    date = np.zeros((len(df_intakeoutput.index),), dtype=int)
    df_intakeoutput['date'] = date
    for i in range(0,len(df_intakeoutput.index)):
        temp = df_intakeoutput['intakeoutputoffset'][i]
        if temp < Firstday:
            df_intakeoutput['date'][i] = 1
        elif temp >= Firstday:
            temp = temp - Firstday
            df_intakeoutput['date'][i] = temp // (24*60) + 2
    
    #創建表格及預設值
    #create sheet and default value
    ftName_0 = ['intaketotal', 'outputtotal', 'dialysistotal', 'nettotal']
    ftName = ['patientunitstayid']
    space_row = [patientunitstayid]
    for i in range(len(ftName_0)):
        for j in range(0,30):
            if j >= int(lenofstay):
                space_row.append(-0.001)
            else:
                space_row.append(-0.001)
    if df_intakeoutput.size == 0:
        df_IO = df_IO.append(pd.Series(space_row, index=df_IO.columns), ignore_index=True)
    else:
        df_IO = df_IO.append(pd.Series(space_row, index=df_IO.columns), ignore_index=True)
        df_intakeoutput = df_intakeoutput.drop_duplicates(subset=['intakeoutputoffset'],keep = 'first')
        df_intakeoutput.index=range(0,len(df_intakeoutput.index))
        for i in range(0,len(df_intakeoutput.index)):
            for j in range(0,len(ftName_0)):
                if pd.isna(df_intakeoutput[ftName_0[j]][i]):
                    df_intakeoutput[ftName_0[j]][i] = -0.001 
        
        #將計算好的值放入表格中
        #put the calculated value into sheet and call "Feature_all_value_with_day" function to turning it into specific format
        for i in range(0,4):
            for j in range(0,30):
                if j >= int(lenofstay):
                    break
                else:
                    str_temp = '%s D%02d'%(ftName_0[i],j+1)
                    df_IO[str_temp][th] = Featrue_all_value_with_day(ftName_0[i],j+1,df_intakeoutput)  
    return df_IO

In [6]:
def main():
    start = time.clock()
    #there are four features we need
    ftName_0 = ['intaketotal', 'outputtotal', 'dialysistotal', 'nettotal']
    ftName = ['patientunitstayid']
    for i in range(0,len(ftName_0)):
        for j in range(0,30):
            ftName.append('%s D%02d'%(ftName_0[i], j+1))
    df_IO = pd.DataFrame(columns= ftName)
    th=0
    for i in range(0,len(df)):
        print("%d/%d: %s"%(i,len(df),df['patientunitstayid'][i]),end='\r')
        df_IO = Intakeoutput(df['patientunitstayid'][i], df['Firstday'][i], df['LoS'][i], th, df_IO, ftName_0)
        th=th+1
    df_IO.to_csv(os.getcwd()+'\\intakeoutput.csv',index=False)
    end = time.clock() - start
    print(end)

In [7]:
if __name__ is '__main__':
    main()

c:\users\userpc\miniconda3\envs\243\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


c:\users\userpc\miniconda3\envs\243\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\userpc\miniconda3\envs\243\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2711.6240708999685.0


c:\users\userpc\miniconda3\envs\243\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  app.launch_new_instance()


In [8]:
#mask
df=pd.read_csv(os.getcwd()+'\\intakeoutput.csv')
df= df.mask(df==-0.001)
df= df.mask(df==0)
df.mask(df==' ').to_csv(os.getcwd()+'\\intakeoutput_mask.csv',index=False)

In [9]:
#filtering outliner
df=pd.read_csv(os.getcwd()+'\\intakeoutput_mask.csv')
droplist = []
pid = []

for i in range(0,len(df.index)):
    for j in range(1,30):
        if df['intaketotal D%02d'%j][i] > 20000 or df['intaketotal D%02d'%j][i] < -1:
            df['intaketotal D%02d'%j][i]=np.nan
        if df['outputtotal D%02d'%j][i] > 20000 or df['outputtotal D%02d'%j][i] < -1:
            df['outputtotal D%02d'%j][i]=np.nan
        if df['dialysistotal D%02d'%j][i] > 20000 or df['dialysistotal D%02d'%j][i] < -20000:
            df['dialysistotal D%02d'%j][i]=np.nan
        if df['nettotal D%02d'%j][i] > 20000 or df['nettotal D%02d'%j][i] < -20000:
            df['nettotal D%02d'%j][i]=np.nan

df.mask(df==-0.001).to_csv(os.getcwd()+'\\intakeoutput_mask.csv',index=False)

In [12]:
#Unite the sign by feature
df=pd.read_csv(os.getcwd()+'\\intakeoutput_mask.csv')
for i in range(0,len(df)):
    for j in range(1,31):
        if(df['dialysistotal D%02d'%j][i]>0):
            df['dialysistotal D%02d'%j][i]=-df['dialysistotal D%02d'%j][i]
        if(df['outputtotal D%02d'%j][i]<0):
            df['outputtotal D%02d'%j][i]=-df['outputtotal D%02d'%j][i]
df.to_csv(os.getcwd()+'\\intakeoutput_mask.csv',index=False)

In [16]:
#先前後補
#imputation
df = pd.read_csv(os.getcwd()+'\\intakeoutput_mask.csv')
ftName_0 = ['intaketotal', 'outputtotal', 'dialysistotal']
for i in range(0,len(df)):
    
    print('%d/%d:%s'%(i,len(df),df['patientunitstayid'][i]),end='\r')
    for j in ftName_0:
        current=0
        flag=0
        for day in range(1,31):
            if(not pd.isna(df['%s D%02d'%(j,day)][i]) and flag==0):
                flag=1#find first value of a month
                current=df['%s D%02d'%(j,day)][i]
                #print(current,flag,day)
                break
        if(flag==1):
            #print(123)
            for missing in range(1,day):
                df['%s D%02d'%(j,missing)][i]=current
            flag=2#if there is a null value than put the value of previous day into it
        #print(df['%s D%02d'%('Nasal cannula O2',1)][i])
        if(flag==2):
            for missing in range(2,31):
                if(pd.isna(df['%s D%02d'%(j,missing)][i]) and not pd.isna(df['%s D%02d'%(j,missing-1)][i])):
                    df['%s D%02d'%(j,missing)][i]=df['%s D%02d'%(j,missing-1)][i]
        mean=[]
        if(flag==0):
            for k in range(0,len(df)):
                mean.append(df['%s D01'%j][k])
            for day in range(1,31):
                df['%s D%02d'%(j,day)][i]=np.nanmean(mean)
    for j in range(1,31):
        if(pd.isna(df['nettotal D%02d'%j][i])):
            df['nettotal D%02d'%j][i]=df['intaketotal D%02d'%j][i]-df['outputtotal D%02d'%j][i]+df['dialysistotal D%02d'%j][i]
    j='nettotal'
    current=0
    flag=0
    for day in range(1,31):
        if(not pd.isna(df['%s D%02d'%(j,day)][i]) and flag==0):
            flag=1#find first value of a month
            current=df['%s D%02d'%(j,day)][i]
            #print(current,flag,day)
            break
    if(flag==1):
        #print(123)
        for missing in range(1,day):
            df['%s D%02d'%(j,missing)][i]=current
        flag=2#if there is a null value than put the value of previous day into it
    #print(df['%s D%02d'%('Nasal cannula O2',1)][i])
    if(flag==2):
        for missing in range(2,31):
            if(pd.isna(df['%s D%02d'%(j,missing)][i]) and not pd.isna(df['%s D%02d'%(j,missing-1)][i])):
                df['%s D%02d'%(j,missing)][i]=df['%s D%02d'%(j,missing-1)][i]
df.to_csv('intakeoutput_com.csv',index=False)
                